In [188]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [189]:
# 검색 페이지 url

def make_search_url(start_date, end_date, loc, category, page): # 선택날짜, 장소, 종류
    # 키워드
    start = start_date
    end = end_date
    loc = loc
    food_catecory =  category

    # url 조합
    base_url = f'https://section.blog.naver.com/Search/Post.naver?pageNo={page}&rangeType=PERIOD&orderBy=sim&'
    date = f'startDate={start}&endDate={end}&'
    loc_cate = f'keyword=%23{loc}%20%23{food_catecory}'
    url_layout = base_url + date + loc_cate

    return url_layout

In [190]:
# title, urls 반환

def ele_lst(input_url):
    titles =[]
    urls=[]
    
    sce = Service(executable_path=ChromeDriverManager().install())
    bws = webdriver.Chrome(service=sce)
    bws.get(input_url)

    ele_list = bws.find_elements(By.CSS_SELECTOR, "a.desc_inner > strong > span")
    elements = bws.find_elements(By.CSS_SELECTOR, "a.desc_inner")

    for ele in elements:
        title = ele.find_element(By.CSS_SELECTOR, "strong").text
        url = ele.get_attribute('href')
        titles.append(title)
        urls.append(url)
    
    bws.quit()

    return titles, urls

In [191]:
# 페이지별로 블로그 제목과 URL 가져오기

def get_blog_titles_and_urls(url_layout, start_date, end_date, loc, category, page_range):
    sce = Service(executable_path=ChromeDriverManager().install())
    bws = webdriver.Chrome(service=sce)
    browser = bws.get(url_layout)
    time.sleep(3)  # 페이지가 로드될 때까지 잠시 대기

    titles_urls_list = []

    for page in range(1, page_range + 1):  # 예시로 3페이지까지만 가져오기
        url = make_search_url(start_date, end_date, loc, category, page)
        bws.get(url)
        time.sleep(1)  # 페이지가 로드될 때까지 잠시 대기

        elements = bws.find_elements(By.CSS_SELECTOR, "a.desc_inner")

        titles = []
        urls = []
        
        for ele in elements:
            title = ele.find_element(By.CSS_SELECTOR, "strong").text
            url = ele.get_attribute('href')
            titles.append(title)
            urls.append(url)
        
        titles_urls_list.append((titles, urls))

    bws.quit()  # 브라우저 종료

    return titles_urls_list

In [192]:
# 결과 출력

def page_res(titles_urls_list) :
    for page, (titles, urls) in enumerate(titles_urls_list, start=1):
        print(f"=== Page {page} ===")
        for i in range(len(titles)):
            print(f"{titles[i]}, {urls[i]}")
        print('\n')

#    result = f"{titles[i]}, {urls[i]}"
#    return result

In [193]:
def extract_info(browser, url):
    names = []
    address = []
    
    browser.get(url)
    print('url get done')
    time.sleep(3)
    
    try:
        # iframe으로 전환
        iframe = browser.find_element(By.CSS_SELECTOR, "iframe#mainFrame")
        browser.switch_to.frame(iframe)
        
        # 지도 이름 추출
        name_element = browser.find_element(By.CSS_SELECTOR, "strong.se-map-title")
        name_text = name_element.text
        print(name_text)
        
        # 위치 정보 추출
        address_element = browser.find_element(By.CSS_SELECTOR, "div.se-module.se-module-map-text > a > p")
        address_text = address_element.text

        # 공감 수 추출
        # sympathy_element = browser.find_element(By.CSS_SELECTOR, "#Sympathy223400728539 > div > span > em.u_cnt._count")
        # sympathy_text = sympathy_element.text
        # print('sym_text type is', type(name_text))
        # sympathy.append(sympathy_text)
        return name_text, address_text#, sympathy_text
        
    except Exception as e:
        # print(f"An error occurred: {e}")
        print('error in', browser.current_url)
        names.pop()
        addresses.pop()
        # sympathy.pop()
        pass

    # return name_text, address_text
    

In [194]:
def extract_info_to_df(browser, url, df):
    names = []
    addresses = []

    browser.get(url)
    # print('url get done')
    time.sleep(1)

    try:
        # iframe으로 전환
        iframe = browser.find_element(By.CSS_SELECTOR, "iframe#mainFrame")
        browser.switch_to.frame(iframe)

        # 이름, 위치 추출
        name_element = browser.find_element(By.CSS_SELECTOR, "strong.se-map-title")
        address_element = browser.find_element(By.CSS_SELECTOR, "div.se-module.se-module-map-text > a > p")
        name_text = name_element.text
        address_text = address_element.text

        # 리스트에 저장
        names.append(name_text)
        addresses.append(address_text)

        # 공감 수 추출
        # sympathy_element = browser.find_element(By.CSS_SELECTOR, "#Sympathy223400728539 > div > span > em.u_cnt._count")
        # sympathy_text = sympathy_element.text
        # print('sym_text type is', type(name_text))
        # sympathy.append(sympathy_text)

    except Exception as e:
        print('error in', browser.current_url, end=', ')
        pass

    if(len(names) > 0 and len(addresses) > 0):
        df.loc[len(df)] = [name_text, address_text]
    else:
        print('no such element')
    

In [195]:
def make_count(input_df):
    output_df = pd.DataFrame(columns=['name', 'address', 'count'])
    for index_in, row_in in input_df.iterrows():
        tf = 0

        for index_out, row_out in output_df.iterrows():
            if(row_in[0] == row_out[0]):
                tf = 1
                #row_out.replace(int(row_out[2]) + 1, inplace= True)
                output_df.loc[index_out, 'count'] = int(row_out[2]) + 1

        if(tf == 0):
            output_df.loc[len(output_df)] = [row_in[0], row_in[1], 1]
            pass

    return output_df